In [ ]:
# Installing Tensorflow Backbone first
!pip install --upgrade tensorflow

# Installing Keras Framework
!pip install Keras

# Installing necessary python packages
!pip install scikit-learn
!pip install matplotlib
!pip install scikit-image

In [ ]:
# Check Tensorflow installation
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)

In [ ]:
# Check Keras installation
import keras

In [ ]:
# Import necessary libraries
import numpy as np
import os, glob, shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Training image location
TRAIN_BEN_IMAGE_PATH='./Train/0/'
TRAIN_MAL_IMAGE_PATH='./Train/1/'

# Validation image location
VALID_BEN_IMAGE_PATH='./Valid/0/'
VALID_MAL_IMAGE_PATH='./Valid/1/'

# Testing image location
TEST_BEN_IMAGE_PATH='./Test/0/'
TEST_MAL_IMAGE_PATH='./Test/1/'

# Training image path list
train_ben_image_list = sorted(glob.glob(TRAIN_BEN_IMAGE_PATH + '*.png'))
train_mal_image_list = sorted(glob.glob(TRAIN_MAL_IMAGE_PATH + '*.png'))

# Validation image path list
valid_ben_image_list = sorted(glob.glob(VALID_BEN_IMAGE_PATH + '*.png'))
valid_mal_image_list = sorted(glob.glob(VALID_MAL_IMAGE_PATH + '*.png'))

# Testing image path list
test_ben_image_list = sorted(glob.glob(TEST_BEN_IMAGE_PATH + '*.png'))
test_mal_image_list = sorted(glob.glob(TEST_MAL_IMAGE_PATH + '*.png'))


In [ ]:
# Displaying benign images
ben_images = []
for i in range(5):
    ben_images.append(mpimg.imread(train_ben_image_list[i]))

plt.figure(figsize=(20,10))
columns = 5
for i, image in enumerate(ben_images):
    plt.subplot(len(ben_images) / columns + 1, columns, i + 1)
    plt.imshow(image)

In [ ]:
# Displaying malignant images    
mal_images = []
for i in range(5):
    mal_images.append(mpimg.imread(train_mal_image_list[i]))

plt.figure(figsize=(20,10))
columns = 5
for i, image in enumerate(mal_images):
    plt.subplot(len(mal_images) / columns + 1, columns, i + 1)
    plt.imshow(image)

In [ ]:
# Constants used for training a network
LEARNING_RATE = 0.0001
NUM_EPOCHS = 10
BATCH_SIZE = 32
IMAGE_SIZE = 48
CHANNELS = 3
MODEL_WEIGHTS = './Model_Weights/'

# Import necessary libraries for building/training/predicting convolutional neural network
from keras.models import Model
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback
from keras.models import load_model
from keras import optimizers
from keras import backend as K
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, ReLU, Dense, Activation
from keras.utils.data_utils import Sequence
from keras.utils import np_utils
from sklearn.metrics import classification_report


In [ ]:
# Create generators for feeding images to our network

# initialize the training data augmentation object
trainAug = ImageDataGenerator(rescale=1 / 255.0)

# initialize the validation data augmentation object
valAug = ImageDataGenerator(rescale=1 / 255.0)

# initialize the testing data augmentation object
testAug = ImageDataGenerator(rescale=1 / 255.0)

# initialize the training generator
trainGen = trainAug.flow_from_directory(
    './Train/',
    class_mode='binary',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode='rgb',
    shuffle=True,
    batch_size=BATCH_SIZE)

# initialize the validation generator
valGen = valAug.flow_from_directory(
    './Valid/',
    class_mode='binary',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode='rgb',
    shuffle=False,
    batch_size=BATCH_SIZE)

# initialize the validation generator
testGen = testAug.flow_from_directory(
    './Test/',
    class_mode='binary',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode='rgb',
    shuffle=False,
    batch_size=BATCH_SIZE)

In [ ]:
# Build our own model
# Reference: https://www.pyimagesearch.com/2018/09/10/keras-tutorial-how-to-get-started-with-keras-deep-learning-and-python/

model = Sequential()
inputShape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

# CONV => RELU => POOL layer set
model.add(Conv2D(32, (3, 3), padding='same', input_shape=inputShape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# (CONV => RELU) * 2 => POOL layer set
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# first (and only) set of FC => RELU layers
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# scale final output value from 0 to 1 using sigmoid activation    
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary() 


In [ ]:
# Compile the model
print('Compiling the Model')
optimizer = optimizers.Adam(learning_rate = LEARNING_RATE)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
mc = ModelCheckpoint(mode='max', filepath=MODEL_WEIGHTS + 'Weights_.{epoch:02d}-{val_accuracy:.4f}.h5', 
                     monitor='val_accuracy', save_best_only='True', save_weights_only='True', verbose=1)

total_num_train = len(train_ben_image_list) + len(train_mal_image_list)
total_num_valid = len(valid_ben_image_list) + len(valid_mal_image_list)
total_num_test = len(test_ben_image_list) + len(test_mal_image_list)

steps_per_epoch = np.ceil(total_num_train / BATCH_SIZE).astype("int32")
print('Steps for Training Epoch: ', steps_per_epoch)

validation_steps = np.ceil(total_num_valid / BATCH_SIZE).astype("int32")
print('Steps for Validation Epoch: ', validation_steps)

testing_steps = np.ceil(total_num_test / BATCH_SIZE).astype("int32")
print('Steps for Testing: ', testing_steps)


In [ ]:
# Start Training
print('Training the Model')
history = model.fit_generator(trainGen,
                              steps_per_epoch=steps_per_epoch,
                              validation_data = valGen,
                              validation_steps = validation_steps,
                              epochs=NUM_EPOCHS,
                              callbacks=[mc])

In [ ]:
# Drawing training history
N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, history.history["loss"], label="train_loss")
plt.plot(N, history.history["val_loss"], label="val_loss")
plt.plot(N, history.history["accuracy"], label="train_acc")
plt.plot(N, history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('./Trainig_history')

In [ ]:
# Use the trained network for testing
print("[Testing network...")
testGen.reset()
predIdxs = model.predict_generator(testGen, steps=testing_steps)
print(predIdxs.shape)

In [ ]:
# Convert the prediction values to binary (0 and 1) and show testing results
print(testGen.class_indices)
predIdxs[predIdxs<0.5] = 0
predIdxs[predIdxs>=0.5] = 1
target_names = ['Benign', 'Malignant']
print(classification_report(testGen.classes, predIdxs, target_names=target_names))

In [ ]:
# List of saved models 
saved_weights_list = sorted(glob.glob(MODEL_WEIGHTS + '*.h5'), key=os.path.getmtime)
print(saved_weights_list)
print(saved_weights_list[-1])

In [ ]:
# Load the best model (highest validation result) and use it for testing
model.load_weights(saved_weights_list[-1])
testGen.reset()
predIdxs = model.predict_generator(testGen, steps=testing_steps)

predIdxs[predIdxs<0.5] = 0
predIdxs[predIdxs>=0.5] = 1

target_names = ['Benign', 'Malignant']
print(classification_report(testGen.classes, predIdxs, target_names=target_names))

In [ ]:
# Install one of the latest image classification model (EfficientNet)
!pip install -U efficientnet

In [ ]:
# Load model
import efficientnet.keras as efn 
eff = efn.EfficientNetB0(weights='imagenet', include_top=False, input_shape=inputShape)
eff.summary()

In [ ]:
# Add classification layer (Dense/full-connected layers)
input = eff.input
x = GlobalAveragePooling2D()(eff.output)
print(x.get_shape())
bsize, c = x.get_shape().as_list()
x1 = Dense(c//2, activation='relu')(x)
x2 = Dropout(0.5)(x1)
x3 = Dense(1, activation='sigmoid')(x2)

output = x3
print('Output Shape: ', output.get_shape())

model2 = Model(input, output)
model2.name = 'EffNet_b1'
model2.summary()

In [ ]:
# Freeze all the layers except the Dense/Fully-connected layers
for layer in model2.layers[:-4]:
    layer.trainable = False

In [ ]:
# Compile the model2
MODEL_WEIGHTS2 = './Model_Weights2/'
print('Compiling the Model2')
optimizer = optimizers.Adam(learning_rate = 0.01)
model2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
mc = ModelCheckpoint(mode='max', filepath=MODEL_WEIGHTS2 + 'Frozen_Weights_.{epoch:02d}-{val_accuracy:.4f}.h5', 
                     monitor='val_accuracy', save_best_only='True', save_weights_only='True', verbose=1)

In [ ]:
# Train the model2
trainGen.reset()
valGen.reset()
history = model2.fit_generator(trainGen,
                              steps_per_epoch=steps_per_epoch,
                              validation_data = valGen,
                              validation_steps = validation_steps,
                              epochs=1,
                              callbacks=[mc])

In [ ]:
# After the Dense/Fully-connected layer weights are optimized, fine-tune all the layers again
for layer in model2.layers:
    layer.trainable = True

optimizer = optimizers.Adam(learning_rate = 0.01*0.1)
model2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
mc = ModelCheckpoint(mode='max', filepath=MODEL_WEIGHTS2 + 'Unfrozen_Weights_.{epoch:02d}-{val_accuracy:.4f}.h5', 
                     monitor='val_accuracy', save_best_only='True', save_weights_only='True', verbose=1)



In [ ]:
# Train the model2 again
trainGen.reset()
valGen.reset()
history = model2.fit_generator(trainGen,
                              steps_per_epoch=steps_per_epoch,
                              validation_data = valGen,
                              validation_steps = validation_steps,
                              epochs=NUM_EPOCHS,
                              callbacks=[mc])

In [ ]:
# List of saved models 
saved_weights_list = sorted(glob.glob(MODEL_WEIGHTS2 + '*.h5'), key=os.path.getmtime)
print(saved_weights_list)
print(saved_weights_list[-1])

In [ ]:
# Load the best model (highest validation result) and use it for testing
model2.load_weights(saved_weights_list[-1])
testGen.reset()
predIdxs = model2.predict_generator(testGen, steps=testing_steps)

predIdxs[predIdxs<0.5] = 0
predIdxs[predIdxs>=0.5] = 1

target_names = ['Benign', 'Malignant']
print(classification_report(testGen.classes, predIdxs, target_names=target_names))